In [1]:
!antlr4 -Dlanguage=Python3 BraKet.g4 -visitor -no-listener

warning(180): BraKet.g4:11:7: chars ' used multiple times in set [',']
warning(180): BraKet.g4:12:10: chars ' used multiple times in set ['\r'? '\n']
warning(180): BraKet.g4:12:10: chars ' used multiple times in set ['\r'? '\n']
warning(180): BraKet.g4:12:10: chars ' used multiple times in set ['\r'? '\n']


In [2]:
import sys
from antlr4 import *
from BraKetLexer import BraKetLexer
from BraKetParser import BraKetParser

def main(show_tokens=True, code=""):
    # 1. Provide the input
    input_stream = InputStream(code)

    # 2. Lexical Analysis
    lexer = BraKetLexer(input_stream)
    token_stream = CommonTokenStream(lexer)

    # 3. Parsing
    parser = BraKetParser(token_stream)
    tree = parser.program()  # Ensure 'prog' matches your .g4 start rule

    # 4. Print the Tree
    print("--- Parse Tree ---")
    print(tree.toStringTree(recog=parser))
    print()

    # 5. Output Tokens (Optional)
    if show_tokens:
        print("--- Tokens ---")
        # We fill the stream to ensure all tokens are loaded
        token_stream.fill()
        
        print(f"{'INDEX':<6} | {'TEXT':<10} | {'TYPE':<15} | {'LINE':<5} | {'COL':<5}")
        print("-" * 50)
        
        for i, token in enumerate(token_stream.tokens):
            # Map the integer type ID to the name you defined in the .g4 file
            t_type = BraKetParser.symbolicNames[token.type] if token.type != -1 else "EOF"
            
            print(f"{i:<6} | {token.text:<10} | {t_type:<15} | {token.line:<5} | {token.column:<5}")

if __name__ == '__main__':
    main()

--- Parse Tree ---
program

--- Tokens ---
INDEX  | TEXT       | TYPE            | LINE  | COL  
--------------------------------------------------
0      | <EOF>      | EOF             | 1     | 0    


line 1:0 mismatched input '<EOF>' expecting {'op', 'main', NEWLINE, 'from', 'import', 'const'}


In [3]:
code='from _mamamo_ import _puke, test'

In [4]:
main(code=code)

--- Parse Tree ---
(program (import_list (import_statement from _mamamo_ import (func_list _puke))) ,  test)

--- Tokens ---
INDEX  | TEXT       | TYPE            | LINE  | COL  
--------------------------------------------------
0      | from       | FROM            | 1     | 0    
1      | _mamamo_   | IDENTIFIER      | 1     | 5    
2      | import     | IMPORT          | 1     | 14   
3      | _puke      | IDENTIFIER      | 1     | 21   
4      | ,          | <INVALID>       | 1     | 26   
5      | test       | IDENTIFIER      | 1     | 28   
6      | <EOF>      | EOF             | 1     | 32   


line 1:26 mismatched input ', ' expecting {'op', 'main', NEWLINE, 'const'}
